## 目标

1. 使用 transformers 的 tokenizers 模块从头开始训练一个分词器。
2. 将语料直接加载到内存中，避免文件 I/O。

## 依赖项
确保你已经安装了 transformers 和 tokenizers 库：

In [ ]:
# pip install transformers tokenizers

## 代码实现

以下代码将：

1. 加载语料到内存。
2. 使用 tokenizers 模块训练一个 BPE 分词器，完全在内存中操作。
3. 保存训练好的分词器，并测试分词效果。

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import time

# 1. 加载语料到内存
def load_corpus_to_memory(file_path):
    start_time = time.time()
    with open(file_path, "r", encoding="utf-8") as f: 
        lines = f.readlines()
    print(f"已加载{len(lines)}行到内存，耗时{time.time() - start_time:.2f}秒")
    return lines

# 2. 训练BPE分词器（完全在内存中）
def train_bpe_tokenizer(corpus, vocab_size=8000, special_tokens=["[PAD]","[UNK]","[CLS]", "[SEP]", "[MASK]"]): 
    start_time = time.time()
    
    # 初始化一个BPE分词器
    tokenizer = Tokenizer(models.BPE())
    
    # 设置预分词器（按空格和标点分词）
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    
    # 定义BPE训练器
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=special_tokens,
        min_frequency=1,# 最小频率，低于此频率的 token 不会加入词汇表
        show_progress=True # 显示训练进度
    )
    
    # 直接在内存中训练
    tokenizer.train_from_iterator(corpus, trainer)
    
    print(f"BPE 分词器训练完成，耗时 {time.time() - start_time:.2f} 秒")
    return tokenizer

# 3. 保存分词器
def save_tokenizer(tokenizer, path="my_tokenizer"): 
    tokenizer.save(path)
    print(f"分词器已保存到{path}")
    
# 4. 分词测试
def tokenize_text(tokenizer, text):
    encoded = tokenizer.encode(text)
    return encoded.tokens 

# 5. 主函数
def main(): 
    # 加载语料到内存
    corpus = load_corpus_to_memory("corpus.txt")
    
    # 为了快速体验，只用前5万行
    # corpus = corpus[:500000]
    print(f"使用 {len(corpus)} 行进行训练")
    
    # 训练BPE分词器
    tokenizer = train_bpe_tokenizer(corpus, vocab_size=32000)
    
    # 保存分词器
    save_tokenizer(tokenizer, "my_tokenizer.json")
    
    # 测试分词效果
    text = "Hello, world!你好，世界！"
    tokens = tokenize_text(tokenizer, text)
    print("\n分词结果：")   
    print(f"混合文本: {tokens}")  
    
if __name__ == "__main__":
    main()
    

已加载2000000行到内存，耗时0.42秒
使用 2000000 行进行训练



BPE 分词器训练完成，耗时 29.93 秒
分词器已保存到my_tokenizer.json

分词结果：
混合文本: ['Hello', ',', 'world', '!', '你好', '，', '世界', '！']


---

## 代码说明

1. **加载语料到内存**
- load_corpus_to_memory() 将 corpus.txt 读入内存，存储为一个字符串列表。
- 500 MB 的文件加载到内存通常只需不到 1 秒。

2. **训练 BPE 分词器**

- 初始化分词器：Tokenizer(models.BPE()) 创建一个基于 BPE 算法的分词器。
- 预分词器：pre_tokenizers.Whitespace() 按空格和标点进行预分词，确保英文按词拆分，中文按字符处理。
- 训练器：BpeTrainer 定义了训练参数：
  - `vocab_size=8000`：目标词汇表大小。
  - `special_tokens`：添加特殊 token（如 [PAD]、[UNK] 等），适合后续与预训练模型结合。
  - `min_frequency=2`：最低频率，减少低频 token。
  - `show_progress=True`：显示训练进度。
- 内存训练：train_from_iterator() 直接从内存中的 corpus（字符串列表）训练分词器，不需要写入文件。

3. **保存分词器**
- `save_tokenizer()` 将训练好的分词器保存为 JSON 文件（`my_tokenizer.json`），可以后续加载使用。

4. **分词测试**
- tokenize_text() 使用训练好的分词器对文本进行分词，返回 token 列表。

---


### 预期效果
- **内存加载**：500 MB 文件加载到内存只需不到 1 秒。
- **训练时间**：5 万行数据，vocab_size=8000，在 20 核心 CPU 上可能只需 1-2 分钟（tokenizers 底层是用 Rust 实现的，效率很高）。
- **CPU 使用率**：tokenizers 会自动利用多线程，CPU 使用率应该较高（接近 100%）。
- **分词结果**：训练完成后，你会得到一个分词器，可以处理中英文混合文本。

**示例输出**

```bash
已加载 1000000 行到内存，耗时 0.85 秒
使用 50000 行进行训练
BPE 分词器训练完成，耗时 90.32 秒
分词器已保存到 my_tokenizer.json

分词结果：
混合文本: ['Hello', ',', 'world', '!', '你好', '，', '世界', '！']
```

---

## 与 SentencePiece 的对比

- 内存训练：tokenizers 支持直接从内存中的字符串列表训练（train_from_iterator），完全避免了文件 I/O，而 SentencePiece 要求输入文件路径。
- 速度：tokenizers 底层是用 Rust 实现的，训练速度通常比 SentencePiece 快，尤其是在多线程场景下。
- 分词效果：两者都使用 BPE 算法，分词结果类似，但 tokenizers 提供了更多灵活性（比如支持多种预分词器、后处理规则等）。
- 生态支持：tokenizers 是 transformers 生态的一部分，训练好的分词器可以直接与 transformers 的模型（如 BERT、T5）结合使用。

---

## 加载和使用训练好的分词器
如果你想在后续任务中加载这个分词器，可以用以下代码：

In [7]:
from transformers import PreTrainedTokenizerFast

# 加载训练好的分词器
tokenizer = PreTrainedTokenizerFast(tokenizer_file="my_tokenizer.json")

# 分词
text = "Hello, world! 你好，世界！"
tokens = tokenizer.tokenize(text)
print(tokens)

# 编码为 ID（适合模型输入）
encoded = tokenizer(text, return_tensors="pt")
print(encoded["input_ids"])

['Hello', ',', 'world', '!', '你好', '，', '世界', '！']
tensor([[14914,    16, 10314,     5, 13842,  9224,  9878,  9213]])
